In [3]:
import sys
sys.path.insert(0,"/DATA_17/hjjo/YOLOX_pruning/YOLOX")

import os
import time
from loguru import logger

import cv2

import torch

from yolox.data.data_augment import ValTransform
from yolox.data.datasets import COCO_CLASSES
from yolox.exp import get_exp
from yolox.utils import fuse_model, get_model_info, postprocess, vis


class YoloDetector:
    def __init__(self):
        self.exp = get_exp(None, "yolox-l")
        self.num_classes = self.exp.num_classes = 1
        self.confthre = 0.5
        self.nmsthre = 0.3
        self.test_size = self.exp.test_size
        self.rgb_means = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)
    

    def load(self):
        self.model = self.exp.get_model()
        logger.info("Model Summary: {}".format(get_model_info(self.model, self.exp.test_size)))
#         self.model.cuda()
#         self.model.eval()
        
        ckpt = torch.load("bytetrack_l_mot17.pth", map_location="cpu")
#         # load the model state dict
        self.model.load_state_dict(ckpt["model"])
  
        self.model.to('cuda').eval()
        print ('load model..')

    
max_batch_size = 1
input = torch.randn(1,3, 640, 640).cuda()


yd = YoloDetector()
yd.load()
model = yd.model
output = model(input)

print(output)


## pytorch to onnx 
torch.onnx.export(
    model,                                # model being run
    input,    # model input (or a tuple for multiple inputs)
    "bytetrack_l_mot17.onnx", # where to save the model (can be a file or file-like object)
    verbose=True, 
    opset_version=11,
    input_names = ['input'],              # the model's input names
    output_names = ['output'],
    dynamic_axes = {'input' : {0 : 'batch_size'},
                    'output' : {0 : 'batch_size'}}
)            # the model's output names


2022-06-10 15:58:36.270 | INFO     | __main__:load:31 - Model Summary: Params: 54.15M, Gflops: 155.31


load model..
tensor([[[-4.7736e+01, -6.7474e+01,  1.5946e+02,  3.6284e+02,  6.0844e-04,
           3.8105e-01],
         [-2.2112e+01, -9.7488e+01,  1.4240e+02,  3.2811e+02,  5.4739e-04,
           4.5459e-01],
         [ 1.9345e+01, -1.0129e+02,  1.2017e+02,  2.6003e+02,  1.2749e-04,
           4.7810e-01],
         ...,
         [ 5.7250e+02,  9.1716e+02,  4.2776e+02,  8.1699e+02,  1.1881e-03,
           4.7266e-01],
         [ 6.7327e+02,  9.3911e+02,  5.5010e+02,  1.0707e+03,  1.3375e-03,
           5.1684e-01],
         [ 7.6060e+02,  8.7496e+02,  4.8273e+02,  9.7605e+02,  3.7934e-03,
           4.1026e-01]]], device='cuda:0', grad_fn=<AsStridedBackward0>)
graph(%input : Float(*, 3, 640, 640, strides=[1228800, 409600, 640, 1], requires_grad=0, device=cuda:0),
      %head.cls_preds.0.weight : Float(1, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=1, device=cuda:0),
      %head.cls_preds.0.bias : Float(1, strides=[1], requires_grad=1, device=cuda:0),
      %head.cls_preds.1.weigh